# COMP90086 Workshop 5

***
In this worksheet, we'll implement a Multilayer perceptron and convolutional neural network (CNN) in Keras—a high-level API for deep learning. We will use MNIST Dataset for experiments.


Table of Contents

- MNIST dataset

- Multilayer Perceptron
     
- Convolutional neural network with standard 2D convolution

- Visualize Filters and Feature Maps in CNN
     
- Bonus: Convolutional neural network with depthwise separable convolution
    

In [ ]:
# Imports
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model

import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow version: ", tf.__version__)

## 1. MNIST dataset

MNIST is a dataset that consists of images of handwritten digits:
* the input data are images of handwritten digits (28×28 pixels with a single 8-bit channel)
* the target is a label in the set $\{0, 1, \ldots, 9\}$

The data is already split into training and test sets. The training set contains 60,000 instances and the test set contains 10,000 instances.

<center>Sample images from MNIST test dataset. (MNIST refers to *Modified National Institute of Standards and Technology*.)</center>

<img style="float: ;" src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png" width=500 height=500>

(Ref & Image Source: Wikipedia [MNIST database](https://en.wikipedia.org/wiki/MNIST_database))

Below we load the data into NumPy arrays using a built-in function from Keras.

Keras is an open-source deep learning library written in Python.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
print("train_images shape:", train_images.shape)
print("test_images shape:", test_images.shape)

Before using the data for classification, we need to do some basic pre-processing: 
 * rescale the images so that each pixel is represented as a float between 0 and 1
 * tranform the input to be a 4D input with number of samples + (rows, cols, channels), so that it can be fed to a layer in keras.

In [ ]:
# Data type adjustment & rescale 
train_images = train_images.astype(float) / 255 #scale the pixel values to be 0-1
test_images = test_images.astype(float) / 255

### <center>Small detour: One reason for why to normalize data inputs</center>

<img style="float: ;" src="https://i.stack.imgur.com/1AxJq.png" width=700 height=700>

(Image Source: JEREMY JORDAN, [Normalizing your data](https://www.jeremyjordan.me/batch-normalization/))

### Exercise 1

tranform the input to be a 4D input with number of samples + (rows, cols, channels)

***hint: use [ny.expand_dims](https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html)***

In [ ]:
#transform training data
#your code


#transform testing data
#your code



print("train_images shape:", train_images.shape)
print("test_images shape:", test_images.shape)

The code block below visualises random examples from the training set.

In [ ]:
num_images = 10
fig, axes = plt.subplots(figsize=(1.5*num_images, 1.5), ncols=num_images)

# replace: Whether the sample is with or without replacement. 
# Default is True, meaning that a value of a can be selected multiple times.
sample_ids = np.random.choice(train_images.shape[0], size=num_images, replace=False)

for i, s in enumerate(sample_ids):
    axes[i].imshow(train_images[s,:,:,0], cmap='binary')
    axes[i].set_title("$y = {}$".format(train_labels[s]))
    axes[i].axis('off')
plt.show()

### <center>Small detour: Choosing Colormaps in Matplotlib</center>

<img style="float: ;" src="https://matplotlib.org/3.4.3/_images/sphx_glr_colormaps_003.png" width=500 height=500>

(Image Source: [Choosing Colormaps in Matplotlib](https://matplotlib.org/devdocs/tutorials/colors/colormaps.html))

Finally, we note that the training set is relatively balanced—there are roughly 6000 examples for each digit.

In [ ]:
plt.hist(train_labels, bins=range(11), align='left')
plt.xticks(ticks=range(11))
plt.title('Distribution of classes in training data')
plt.ylabel('Frequency')
plt.xlabel('Digit')
plt.show()

## 2. Multilayer perceptron

The handwritten digit recognition task is an example of a _multi-class_ classification problem. 
There are 10 classes—one for each digit $0, 1,\ldots, 9$.

We need to define 10 output units in the output layer and apply a softmax activation function to generate probability distribution of the classes


This MLP model can be expressed in Keras as follows.

In [ ]:
# Create the MLP model
mlp = keras.Sequential(
    [
        layers.Input((28,28,1)),               # Tell Keras the shape of the input array (a single-channel 28×28 image)
        layers.Flatten(),                      # Unravel/flatten the input array
        layers.Dense(16, activation='relu'),   # Add a fully-connected layer with 16 units and ReLU activation function as the hidden layer
        layers.Dense(10, activation='softmax') # Add a fully-connected layer with a softmax activation function
    ]
)


### <center>Small detour: Softmax & ReLU</center>
<img style="float: left ;" src="https://pic3.zhimg.com/80/v2-998ddf16795db98b980443db952731c2_1440w.jpg?source=1940ef5c" width=400 height=400>

(Image Source: These images appears in many places, including [here](https://www.programmersought.com/article/58194026613/))


<img style="float: right;" src="https://miro.medium.com/max/357/1*oePAhrm74RNnNEolprmTaQ.png" width=300 height=300>

(Image Source: [ReLU](https://medium.com/@kanchansarkar/relu-not-a-differentiable-function-why-used-in-gradient-based-optimization-7fef3a4cecec))

Next, we have to specify a loss function. We use categorial [cross_entropy](https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class) for this task. We use [Adam](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam) (Adaptive Moment Estimation) as the optimisation algorithm to update weights of the network and directs Keras to keep track of accuracy during training.

In [ ]:
# Configure the model
mlp.compile(optimizer='adam',
           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), #use SparseCategoricalCrossentropy because labels are integers. If the labels are one-hot representation, please use CategoricalCrossentropy loss.
           metrics=['accuracy'])


We're now ready to fit the `mlp` model using the training data. 
By setting `batch_size = 100`, each gradient descent step is computed w.r.t. a random batch of 100 training instances.
By setting `epochs = 20`, we loop over the complete training data 20 times.

In [ ]:
# Start training
history_mlp = mlp.fit(train_images, train_labels, epochs=20, batch_size=100, 
                      validation_split=0.2)

The plots below show that the model fit is unlikely to improve significantly with further training. 
Both the test loss and accuracy have flattened out.

In [ ]:
plt.plot(history_mlp.history['accuracy'], label='Train')
plt.plot(history_mlp.history['val_accuracy'], label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Training and validating accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history_mlp.history['loss'], label='Train')
plt.plot(history_mlp.history['val_loss'], label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('Loss')
plt.title('Training and validating loss')
plt.legend()
plt.show()

However, all of these observations were based on validation data. The best way to test it is to use the actual data generated earlier:

### Evaluate model performance

In [ ]:
# Test the model after training
test_results = mlp.evaluate(test_images, test_labels)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

Similar result? That's great :)
### Make a prediction

***Hint: Try refreshing several times to see different test examples***

In [ ]:
sample_id = np.random.choice(test_images.shape[0])
digit = test_images[sample_id]

plt.imshow(digit.squeeze(), cmap="binary")
plt.title("$label = {}$".format(test_labels[sample_id]))
plt.axis('off')
plt.show()

# make a prediction for the test sample
digit = np.reshape(digit, (-1, 28*28))

predict_digit = mlp.predict(digit)

for index, i in enumerate(predict_digit[0]):
    print ("probability for the test sample is", index, ":",i)

print("------")   
# The predicted digit (with the highest probability value)
print("The predicted digit:",np.argmax(mlp.predict(digit, verbose=0)))

Notice that we use the Softmax activation function in the output layer, so `predict_digit` returns a 1D tensor with 10 elements, corresponding to the probability values of each category. The predicted digit has the highest probability value.

### Summary

We can get a useful summary, including output shape and number of parameters  of the model architecture using the `summary` method, as shown below.
* Try to  compute the parameters of each layer yourself. Note that if you donot count bias, there could be some difference compare with the summary output number (with bias in a layer, the additional NO. parameters is the NO. filters in the layer).

In [ ]:
mlp.summary()

## 3. Convolutional neural network with standard 2D convolution 

Let's now construct a CNN with standard 2D convolution with the following structures for classification on the MNIST dataset.

| Number | Layer type    | Specification                                                                           | Keras function |
|--------|---------------|-----------------------------------------------------------------------------------------|----------------|
| 1      | Convolutional | 8 5×5 filters with a stride of 1 and a ReLU activation function                         | Conv2D         |
| 2      | Pooling       | Max pooling with a 2×2 filter and a stride of 2 (implies pooled regions do not overlap) | MaxPooling2D   |
| 3      | Convolutional | 16 5×5 filters with a stride of 1 and a ReLU activation function                        | Conv2D         |
| 4      | Pooling       | Same specs as pooling layer #1                                                          | MaxPooling2D   |
| 5      | Flatten       | Nil                                                                                     | Flatten        |        |
| 6      | Dense         | 10 units (one for each target class) with a softmax activation function.                | Dense          |



### Exercise 2

Complete the code block below to instantiate the model in Keras. 

Hint: check keras documents for usages of the layers, e.g., how to set the hyperparameters.

*2D convolutional layer: [layers.Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/)

*2D Maxpooling layer: [layers.MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/)


In [ ]:
cnn = keras.Sequential(
    [
        layers.Input((28, 28, 1)),
        
        #
        #your code
        #
        
        
        layers.Flatten(),
        layers.Dense(10, activation='softmax')
    ], 
)

Compile and training the model as before

In [ ]:
cnn.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=['accuracy'])

Training the `cnn` model takes longer than training the `mlp` model on a CPU. 
You may like to set the number of epochs to a smaller number (e.g. `epochs=10`) if you don't have much time to spare.

In [ ]:
history_cnn = cnn.fit(train_images, train_labels, epochs=10, batch_size=100, 
                      validation_split=0.2)

Let's plot the accuracy and loss for each epoch

In [ ]:
plt.plot(history_cnn.history['accuracy'], label='Train')
plt.plot(history_cnn.history['val_accuracy'], label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Training and validating accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history_cnn.history['loss'], label='Train')
plt.plot(history_cnn.history['val_loss'], label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('Loss')
plt.title('Training and validating loss')
plt.legend()
plt.show()

We can get a useful summary, including output shape and number of parameters  of the model architecture using the `summary` method, as shown below.
* Try to  compute the parameters of each layer yourself. Note that if you donot count bias, there could be some difference compare with the summary output number (with bias in a layer, the additional NO. parameters is the NO. filters in the layer).

In [ ]:
cnn.summary()


***
**Question:** How does  `cnn` model compare with  the earlier `mlp` model in terms of number of parameters and testing accuracy ?
***


## 4. Visualize Filters and Feature Maps in CNN

We will discover how to develop simple visualizations for filters and feature maps in a CNN. We'll be using the [VGG16 model](https://keras.io/api/applications/vgg/#vgg16-function) provided by Keras, trained on the ImageNet dataset. For more information on the VGG16, see the [paper](https://arxiv.org/abs/1409.1556).

This section draws heavily on [Jason Brownlee](https://machinelearningmastery.com/)‘s work.

### Load a VGG16 model loaded with pre-trained ImageNet weights

If this is the first time that you have loaded the model, the weights will be downloaded from the internet and may take a moment to download depending on the speed of your internet connection.

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

# load the model
model = VGG16()
# summarize the model
model.summary()

Now that we have a pre-trained model, we can use it as the basis for visualizations.

### Visualize Filters

#### Summarize the model filters
We can access all of the layers of the model via the `model.layers` property.

Each layer has a `layer.name` property, where the convolutional layers have a naming convolution like `block#_conv#`, where the `#` is an integer. Therefore, we can check the name of each layer and skip any that don’t contain the string `conv`.

Each convolutional layer has two sets of weights. One is the block of filters and the other is the block of bias values. These are accessible via the `layer.get_weights()` function. We can retrieve these weights and then summarize their shape.

In [ ]:
# summarize filter shapes
for layer in model.layers:
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    # get filter weights
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape)

We can see that all convolutional layers use 3×3 filters.

We can see that for the input image with three channels for red, green and blue, each filter has a depth of three (here we are working with a channel-last format). We could visualize one filter as a plot with three images, one for each channel, or compress all three down to a single colour image, or even just look at the first channel and assume the other channels will look the same. The problem is that we have 63 other filters we might like to visualize.

We can retrieve the filters from the first layer as follows:

In [ ]:
# retrieve weights from the second hidden layer
filters, biases = model.layers[1].get_weights()

The weight values will likely be small positive and negative values centered around 0.0.

We can normalize their values to the range 0-1 to make them easy to visualize.

In [ ]:
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

We will be visualising the first six filters out of 64 filters from 1st layer, and plot each of the three channels of each filter. One row for each filter and one column for each channel.

In [ ]:
# plot first few filters
n_filters, ix = 6, 1

fig = plt.figure(figsize=(8, 12))
for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(3):
        # subplot for 6 filters and 3 channels
        # specify subplot and turn of axis
        ax = plt.subplot(n_filters, 3, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(f[:, :, j], cmap='gray')
        ix += 1
# plot the filters
plt.show()

We can see that in some cases, the filter is the same across the channels (the first row), and in others, the filters differ (the last row).

The dark squares indicate small or inhibitory weights and the light squares represent large or excitatory weights. Using this intuition, we can see that the filters on the first row detect a gradient from light in the top left to dark in the bottom right.


### Visualize Feature Maps

The activation maps, called feature maps, capture the result of applying the filters to input, such as the input image or another feature map.

We need a clearer idea of the shape of the feature maps output by each of the convolutional layers and the layer index number so that we can retrieve the appropriate layer output. The block below will enumerate all layers in the model and print the output size or feature map size for each convolutional layer as well as the layer index in the model.

In [ ]:
# summarize feature map shapes
for i in range(len(model.layers)):
    layer = model.layers[i]
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    # summarize output shape
    print(i, layer.name, layer.output.shape)

We can use this information and design a new model that is a subset of the layers in the full VGG16 model. The model would have the same input layer as the original model, but the output would be the output of a given convolutional layer, which we know would be the activation of the layer or the feature map.

For example, after loading the VGG model, we can define a new model that outputs a feature map from the first convolutional layer (index 1) as follows.

In [ ]:
# redefine model to output right after the first hidden layer
model = Model(inputs=model.inputs, outputs=model.layers[1].output)

After defining the model, we need to load the image with the size expected by the model, in this case, 224×224.

### Exercise 3: Try replacing it with the image of your choice
Download your favourite photo and place it in your current working directory with the filename `harry.png`.

In [ ]:
# load the image with the required shape
img = load_img('./harry.png', target_size=(224, 224))

plt.imshow(img)
plt.title("Harry (Photographed by Jiayang Ao)")
plt.axis('off')
plt.show()

# convert the image to an array
img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
img = np.expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
img = preprocess_input(img)

We are now ready to get the feature map. We can do this easy by calling the `model.predict()` function and passing in the prepared single image.

We know the result will be a feature map with 224x224x64. We can plot all 64 two-dimensional images as an 8×8 square of images.

In [ ]:
# get feature map for first hidden layer
feature_maps = model.predict(img)


fig = plt.figure(figsize=(15,15))
# plot all 64 maps in an 8x8 squares
square = 8
for i in range(1, square*square+1):
    # specify subplot and turn of axis
    ax = plt.subplot(square,square,i)
    ax.set_xticks([])
    ax.set_yticks([])
    # plot feature maps in grayscale
    plt.imshow(feature_maps[0,:,:,i-1] , cmap='gray')

# plot all feature maps   
plt.show()

## Bonus: Convolutional neural network with depthwise separable convolution 

### Bonus Exercise 

Construct a CNN with the previous architecture (same output size of feature maps in each layer), except that the standard 2D convolution is replaced  with depthwise separable convolution 

Complete the code block below to instantiate the model in Keras. 

***Hint: check keras documents for usages of the layers, e.g., how to set the hyperparameters.***

- SeparableConv2D layer: [layers.SeparableConv2D](https://keras.io/api/layers/convolution_layers/separable_convolution2d/)


In [ ]:
cnn_ds = keras.Sequential(
    [
        layers.Input((28, 28, 1)),
        
        #
        #your code
        #
    
        layers.Flatten(),
        layers.Dense(10, activation='softmax')
    ], 
)

Compile and training as before

In [ ]:
cnn_ds.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=['accuracy'])

history_cnn_ds = cnn_ds.fit(train_images, train_labels, epochs=5, batch_size=100, validation_split=0.2)

We can get a useful summary, including output shape and number of parameters  of the model architecture using the `summary` method, as shown below.
* Try to  compute the parameters of each layer yourself. Note that if you donot count bias, there could be some difference compare with the summary output number (with bias in a layer, the additional NO. parameters is the NO. filters in the layer).

In [ ]:
cnn_ds.summary()

***
**Question:** How does  `cnn_ds` model compare with  the earlier `cnn` model in terms of number of parameters and testing accuracy ?
***
